# High velocity food transport: a study in projectile cattle
### Jonathan Kelley and Solomon Greenberg

![he](images/flyingcow.jpg)

2018 has been a crazy year for technology. Smartphones are up to 89.6% screen-to-body ratio. Fidget spinners took over the world by storm. IoT-connected toilets now track your bowel movements and let you share it with your friends. 

Despite the incredible advances in humanity's cutting edge, the field of high velocity food transport remains nearly silent. However, Elon Musk's proposed rocket transport concepts might indeed change the farm-to-fork experience for our daily bread.  

While Musk might understand the logistics of supply chain and have a degree in engineering, he is still thinking "inside-the-box." We see the most efficient high-velocity food transport system completely void of rockets - fuel is expensive, big explosions are common, and it shouldn't take several million dollars to launch your next Big Mac from SanFran to Boston. Our goal: launch cattle across the state of Massachussets without a rocket. 


A big catapult should do the trick.

Ideally, our beachhead market would be the type of cattle that is most efficient to launch - our catapult can be cheaper and we can launch higher quantities. At least that's what the "suits upstairs" said.

Anyways, we need to figure out which cattle would fly the farthest for a fixed amount of weight.

To maintain consistency, we will be launching ~630 kg of each animal with a single shot

Our animals:

| Animal         | Mass (kg) | Quantity Per Launch |
|----------------|-----------|---------------------|
| Chicken        | 6.35      | 99                  |
| Cow            | 630.49    | 1                   |
| Goat           | 124.74    | 5                   |
| Pig            | 127.91    | 5                   |
| Sheep          | 113.31    | 5                   |
| Fish (Tilapia) | 2.27      | 228                 |

sidebar: a fishing vessel that can launch the fish back to shore is actually a decent business idea.

### Robust and Versatile!

We'd love to be able to launch any plane/train/animal that our heart so desires, so we need a robust physics model. The four fundamental forces of the universe include:
- Strong force
- Weak Force
- Gravity
- Electromagnetism

Obviously, this is too general for us to write a single "net force" function, so our model will allow users to input their own force function that returns a force from the object's state.

We also need some understanding of the object's aerodynamic properties (surprisingly falls under electromagnetism).
We could either generalize the flight physics or write our own 2D computational fluid dynamics solver.

We wrote our own CFD solver to handle images (emoji too)!

In [1]:
from modsim_cfd import image2cdf, emoji2cdf

### Behind the scenes

The CFD solver takes the input image and converts it to a 2D array. We can then solve the Navier-Stokes fluid equations where the left-most boundary conditions sets the air flow at fixed initial momentum.

By analyzing the momemntum drop of the fluid as it is solved around the input image, we can get values for the coefficients of drag and lift. More analysis can determine other characteristics like skin friction and form friction, but we are assuming turbulent fluid flow regardless of the object shape. 

![Example of Navier-Stokes solved around a pig](images/pignavier.png)

### Building a projectile model

We can treat each animal as a rigid body with a mass, position, velocity, and net acceleration.

Each animal also has the coefficients of drag and lift that we determined with the CFD solver.

In [3]:
class projectile:
    def __init__(self, image, mass, dt = .1, is_emoji = False):
        '''
        Initalize a template projectile with basic 2D properties.
        This function requires a profile (image) to be passed in as a string.
        This string can either be the file location to the image or the unicode for an emoji.
        Make sure to set the is_emoji parameter to true when relevant.
        '''
        self.state.position = np.array([0,0])
        self.positions = []
        
        self.state.velocity = np.array([0,0])
        self.velocities = []

        self.state.acceleration = np.array([0,0])
        self.accelerations = []
        
        self.state.dt = dt
        self.times = []
        
        self.state.mass = mass
        
        self.is_emoji = is_emoji
        self.define_aerodynamics(self.is_emoji)
        self.state.cl = self.cl
        self.state.cd = self.cd
        
    def define_aerodynamics(self, is_emoji = False):
        '''
        Using the modsim_cdf package, we can generate relevant aerodynamic properties of whatever image we want.
        This function will use the relevant solver based on the is_emoji flag. 
        The current output properties are the coefficients of lift and drag.
        '''
        if is_emoji:
            aerodynamics = modsim_cdf.emoji2cdf(self.image, 
                                                self.emoji_width, 
                                                properties = ["cd", "cl"])
        else:
            aerodynamics = modsim_cdf.image2cdf(self.image, 
                                                self.image_width, 
                                                properties = ["cd", "cl"])
        self.cd = aerodynamics['coefficient_of_drag']
        self.cl = aerodynamics['coefficient_of_lift']
        self.aerodynamic_properties = aerodynamics['properties']
        
    def update_state(self):#, position = self.position, velocity = self.velocity, acceleration = self.acceleration, dt = self.dt):
        '''
        Updates the state of the object and adds state to state history
        dt can be overrriden for adaptive time step algorithms but defaults to the intial timestep        
        '''
        self.state.position = position
        self.positions.append(position)

        self.state.velocity = velocity
        self.velocities.append(velocity)
                
        self.state.acceleration = acceleration
        self.accelerations.append(acceleration)
        
        self.state.t += self.dt
        self.times.append(self.t)
                

    def get_accel(self):
        '''
        Loops through the associated force functions and returns a net acceleration
        '''
        
        net_force = np.array([0,0,0])
        
        for each_force in self.forces:
            net_force += each_force(self.state)
        
        return(netforce/self.state.mass)

    def tick(self):#, dt = self.dt):
        '''
        Use a custom RK4 method to move the object.
        The default behavior is to use the dt of the object when initalized - this can be overridden with the dt parameter
        The defaukt behavior will also automatically log the state of the object every step, this can be set to 0 for never or n for striping
        '''
        velo = self.velo
    
        k1 = dt * self.get_accel();
        l1 = dt * velo;

        k2 = dt * self.get_accel(0.5 * l1);
        l2 = dt * (velo + (0.5 * k1));

        k3 = dt * self.get_accel(pos + (0.5 * l2) );
        l3 = dt * (velo + (0.5 * k2));

        k4 = dt * self.get_accel(pos + l3);
        l4 = dt * (velo + k3);

        velo += (k1 + (2.0 * k2) + (2.0 * k3) + k4) / 6.0;
        pos += (l1 + (2.0 * l2) + (2.0 * l3) + l4) / 6.0; 
        
        self.update_state(pos = self.position,
                          velocity = self.velocity,
                          acceleration = self.get_accel())

### With the framework above, we can load and launch nearly any object we want with somewhat realistic physics

Our custom module `modsim_cdf` gives us useful information about the aerodynamic properties of whatever object we initialize. For instance, we can learn the drag and lift properties of a chicken from an image pulled from google:

![chicken](images/chicken.png)

In [39]:
chicken = projectile(image = 'images/chicken.png',
                     image_width = 0.6096,  #meters
                     mass = 1.25,          #kilograms
                     dt = .1)

chicken.aerodynamic_properties

TypeError: __init__() got an unexpected keyword argument 'image'


or even from the emoji version (Apple's edition):

🐔


In [ ]:
emoji_chicken = projectile(image = '🐔',
                           image_width = 0.6096,  #meters
                           mass = 1.25,          #kilograms
                           dt = dt)

chicken.aerodynamic_properties

### Launching our chickens.

When launching our animals, we are interested in the fluid interactions of the profiles in air. We also want to keep the kinetic energy of the catapult fixed for each test, so we will evenly distribute the total kinetic energy among each object launched.

To add realistic physics to our simulation, we need to start adding force interactions.

For an object flying through the air, our free-body diagram indicates that drag, lift, gravity, and thrust are relevant forces. We ultimately choose not to include other forces like interplanetary graviation or solar wind because of their minute impact on the simulation results for the current small scale. However, our model is flexible enough to add in these interactions at a later date.

In [5]:
def lift(object_state):
    '''
    Calculates a lift vector normal to the velocity path of the object
    
    L = Cl * A * (0.5 * rho * V^2)
    
    '''
    rho = 1.225 # kg/m^3
    
    perp_velocity = np.array([-object_state.velocity[1], object_state.velocity[0]]) 
    perp_velocity = perp_velocity/perp_velocity.mag
    lift = object_state.cl * object_state.wing_area * rho * 0.5 * (object_state.velocity)**2.0
    
    return perp_velocity * lift

In [6]:
def drag(object_state):
    '''
    Calculates the drag vector
    
    D = Cd * A * (0.5 * rho * V^2)
    
    '''
    rho = 1.225 # kg/m^3
    
    velocity = object_state.velocity * -1
    velocity = velocity / velocity.mag
    drag = object_state.cd * object_state.frontal_area * rho * 0.5 * (object_state.velocity)**2.0
    
    return drag * velocity

In [ ]:
def gravity(object_state):
    '''
    Calculats the force of gravity
    
    F = MG
    '''
    G = 9.8 # m / s ^2
    return object_state.mass * G

In [ ]:
def launch_end_condition(object_state):
    return object_state.position[1] < 0

### Need a launcher

We know have a representation of the physics of flight and a robust way of generated useful aerodynamics data for each object entered into the simulation. However, we don't yet have a system to launch the animals and collect the data.

In [ ]:
class launcher:
    def __init__(self, release_height = 15 ):
        self.release_position = np.array([0, release_height])
        pass
    
    def load_projectile(self, projectile, num_projectiles = 1):
        '''
        Sets the initial conditions for the loaded projectiles
        '''
        self.projectile = projectile
        self.projectile.position = self.release_position
        self.num_projectiles = num_projectiles
        
    def launch_projectile(self, kinetic_energy, launch_angle = np.radians(45)):
        '''
        Set the initial velocity for the projectiles
        Run their "tick()" functions until the end condition is met
        Build the results object for easy data analysis
        '''
        ke_per_projectile = kinetic_energy/self.num_projectiles
        velocity_norm = np.array([np.cos(launch_angle), np.sin(launch_angle)])

        # k = 1/2 m v^2
        # sqrt(2k/m) = v
        
        vel_mag = np.sqrt(2.0 * ke_per_projectile / projectile.mass)
        projectile.velocity = velocity_norm * vel_mag

        pass
    

### Flight data for 99 chickens:

In [1]:
catapult = launcher()

chicken = projectile(image = 'images/chicken.png',
                     mass = 1.25,
                     dt = dt)

chicken.add_force(lift)
chicken.add_force(drag)
chicken.add_force(gravity)

catapult.load(chicken, num_projectiles = 99)
chicken.results = catapult.launch(kinetic_energy = 1e6,
                                  launch_angle = np.radians(45)
                                  end_condition = launch_end_condition)

plt.figure(dpi = 200)
plt.plot(chicken.results['Time'], chicken.results['Position'], label = 'Position vs. Time')

SyntaxError: invalid syntax (<ipython-input-1-fe0e8c49d302>, line 1)

### Flight data for the other animals:

| Animal         | Mass (kg) | Quantity Per Launch |
|----------------|-----------|---------------------|
| Chicken        | 6.35      | 99                  |
| Cow            | 630.49    | 1                   |
| Goat           | 124.74    | 5                   |
| Pig            | 127.91    | 5                   |
| Sheep          | 113.31    | 5                   |
| Fish (Tilapia) | 2.27      | 228                 |

### Launching all the different animals:

In [37]:
animals = {"chicken": {'num_animals' : 99,  'mass' : 6.35,   'image' : 'images/chicken.png'},
           "cow":     {'num_animals' : 1,   'mass' : 630.49, 'image' : 'images/cow.png'},
           "goat":    {'num_animals' : 5,   'mass' : 124.74, 'image' : 'images/goat.png'},
           "pig":     {'num_animals' : 5,   'mass' : 124.74, 'image' : 'images/pig.png'},
           "sheep":   {'num_animals' : 5,   'mass' : 113.91, 'image' : 'images/sheep.png'},
           "tilapia": {'num_animals' : 228, 'mass' : 1.25,   'image' : 'images/tilapia.png'}}
           
for animal, properties in (animals.items()):
    catapult = launcher()

    animal_sim = projectile(image = properties['image'],
                         mass = properties['mass'],
                         dt = dt)

    animal_sim.add_force(lift)
    animal_sim.add_force(drag)
    animal_sim.add_force(gravity)

    catapult.load(animal_sim, num_projectiles = properties['num_animals'])
    animal['results'] = catapult.launch(kinetic_energy = 1e6,
                                        launch_angle = np.radians(45)
                                        end_condition = launch_end_condition)


{'chicken': {'num_animals': 99, 'mass': 6.35, 'image': 'images/chicken.png'}, 'cow': {'num_animals': 1, 'mass': 630.49, 'image': 'images/cow.png'}, 'goat': {'num_animals': 5, 'mass': 124.74, 'image': 'images/goat.png'}, 'pig': {'num_animals': 5, 'mass': 124.74, 'image': 'images/pig.png'}, 'sheep': {'num_animals': 5, 'mass': 113.91, 'image': 'images/sheep.png'}, 'tilapia': {'num_animals': 228, 'mass': 1.25, 'image': 'images/tilapia.png'}}
{'num_animals': 99, 'mass': 6.35, 'image': 'images/chicken.png'}
{'num_animals': 1, 'mass': 630.49, 'image': 'images/cow.png'}
{'num_animals': 5, 'mass': 124.74, 'image': 'images/goat.png'}
{'num_animals': 5, 'mass': 124.74, 'image': 'images/pig.png'}
{'num_animals': 5, 'mass': 113.91, 'image': 'images/sheep.png'}
{'num_animals': 228, 'mass': 1.25, 'image': 'images/tilapia.png'}


### Results:

For each animal, we have position, velocity, and acceleration vs time data. For the sake of this project, we are primarily curious in the greatest horizontal position achieved. The animals with the greatest horizontal position will have traveled the farthest for a fixed amount of kinetic energy and starting mass.

This is given below:

In [ ]:
plt.figure(dpi = 200)
animals_to_plot = []
anomals
plt.bar()

### Interpretations:

### Conclusions: